In [19]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import json

# Data Exploration

## FakeNewsNet

In [20]:
root = '/home/eddyedzwan/code/EddyEdzwan/FakeNewsNet/code/fakenewsnet_dataset/politifact/fake'
dir_ = os.listdir(root)

#Printing the content of a JSON file download for preview
for folder in dir_:
    f = open(os.path.join(root,folder,'news content.json'))
    data = json.load(f)
    print(json.dumps(data, indent=4, sort_keys=True))
    break

{
    "authors": [],
    "canonical_link": "https://web.archive.org/web/20180624033812/http://flashnewss.club/2018/06/11/breaking-shep-smith-to-be-canned-because-he-cant-control-his-hate-for-donald-trump/",
    "images": [
        "https://web.archive.org/web/20180624033812im_/http://flashnewss.club/wp-content/uploads/2018/06/Screenshot_20180617-0102212-80x80.png",
        "https://web.archive.org/web/20180624033812im_/http://flashnewss.club/wp-content/uploads/2018/06/news_m-2.jpg",
        "https://web.archive.org/web/20180624033812im_/http://flashnewss.club/wp-content/uploads/2018/06/80de7608-53f0-460c-8947-b5a7d76720d2-400x240.jpg",
        "https://web.archive.org/web/20180624033812im_/http://flashnewss.club/wp-content/uploads/2018/06/20-24-400x240.png",
        "https://web.archive.org/web/20180624033812im_/http://flashnewss.club/wp-content/uploads/2018/06/few.jpg",
        "https://web.archive.org/web/20180624033812im_/http://2.gravatar.com/avatar/57f977b92bc08c494494dc40ed0ae92b

In [21]:
def getnews(directory):
    '''Function to read news content in given directory using the FakeNewsNet dataset'''
    dictlist = []
    
    #Only get the following columns from the dataset
    cols = ['title','text','authors','num_images','url']
    
    folders = glob.glob(directory+'/*')
    for index, subdir in enumerate(folders):
        file_path = glob.glob(subdir+'/*')
        #check if glob returned a valid file path (non-empty list)
        if len(file_path) == 1:
            file = open(file_path[0]).read()
            jsondata = json.loads(file)
            dictlist.append(scaledict(jsondata))
    
    return pd.DataFrame(dictlist,columns=cols)

In [22]:
def scaledict(ajson):
    '''Function to process json pulled from FakeNewsNet'''
    thedict = {'url':ajson['url'],'title':ajson['title'],'text':ajson['text'],'num_images':len(ajson['images']),'authors':str(ajson['authors'])}
#     ext = tldextract.extract(ajson['url'])
#     thedict['domain'] = ext.domain
    return thedict

In [23]:
#Running the getnews function for the Politifact - Fake news content
root = '/home/eddyedzwan/code/EddyEdzwan/FakeNewsNet/code/fakenewsnet_dataset/politifact/fake'

getnews(root)

,title,text,authors,num_images,url
0,BREAKING: Shep Smith To Be Canned Because He C...,A dispute in Angela Merkel’s coalition over mi...,[],25,https://web.archive.org/web/20180615125007/htt...
1,"CNN Shocks America, FIRES ‘Journalist’ Who Mad...","On Sunday, CNN did something shocking: they to...","['About Janet Farrow', 'Janet Farrow Is An Inv...",5,https://web.archive.org/web/20180502181153/htt...
2,"Nancy Pelosi Suffers Breakdown, Taken From Cap...",It’s not a good day for Democrats.\n\nDemocrat...,"['About Truth Dealer', 'Truth Dealer Is A Cons...",52,https://web.archive.org/web/20170616181605/htt...
3,Libtard Hero Al Franken Forced To Resign In Di...,Libtard Democrat Al Franken will resign his se...,['Please Enter Your Name Here'],10,https://web.archive.org/web/20170619144927/htt...
4,Bill Gates: “I think Donald Trump will go down...,Please Support Online Petition To Congress For...,[],15,https://web.archive.org/web/20170205045142/htt...
...,...,...,...,...,...
392,"Fact-Check: No, Fall River, Massachusetts, Was...",Rep. Joe Kennedy III (D-MA) claimed the host-c...,['Ian Mason'],30,http://www.breitbart.com/live/state-union-fact...
393,"Daily Vine – Dubai, UAE",People when going to sale their house then the...,[],15,daily-vine.com/cnn-pulls-plug/
394,Political Mayhem,Government over-reach has been a big issue for...,[],14,https://politicalmayhem.news/state-minnesota-j...
395,Food Stamp Enrollment Drops by Four Million in...,Four million people dropped off the food stamp...,['Katherine Rodriguez'],30,http://www.breitbart.com/big-government/2018/0...


## Liar Dataset (TBC - Does not contain the data we need, can be placed aside)

In [3]:
dataset = load_dataset('liar')

dataset

Using custom data configuration default
Reusing dataset liar (/home/eddyedzwan/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 10269
    })
    test: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1283
    })
    validation: Dataset({
        features: ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'context'],
        num_rows: 1284
    })
})

In [7]:
X_train = pd.DataFrame(dataset['train'])
X_valid = pd.DataFrame(dataset['validation'])
X_test = pd.DataFrame(dataset['test'])

X_train.head()

,id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context
0,2635.json,0,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,2,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,,,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
